<a href="https://colab.research.google.com/github/purvi202211023/IT555-Deep-Neural-NLP/blob/Train-Word2Vec-on-peS2o-Dataset-(AllenNLP)-_Assignment1/Deep_Neural_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install num2words
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
import re
import inflect
import pandas as pd
import numpy as np
from num2words import num2words  # For numerical-to-text conversion

In [ ]:
# Download necessary NLTK data
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Load the training data
df_train_n = pd.read_pickle("/content/drive/MyDrive/NLP/train_data_n.pkl")

# Load the test data
df_test_n = pd.read_pickle("/content/drive/MyDrive/NLP/test_data_n.pkl")

In [ ]:
# combined_df = pd.concat([df_train_n, df_test_n], ignore_index=True)
# combined_df

In [ ]:

# Create an inflect engine for number-to-text conversion
p = inflect.engine()

def preprocess_text(text):
    # Convert text to lowercase
    text = text.lower()

    # Remove URLs
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)

    # Remove bullets
    text = re.sub(r'\u2022', '', text)

    # Handle apostrophe
    text = re.sub(r"\'s", " is", text)
    text = re.sub(r"\'ve", " have", text)
    text = re.sub(r"can't", "cannot", text)
    text = re.sub(r"n't", " not", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"\'d", " would", text)
    text = re.sub(r"\'ll", " will", text)

    # Handle hyphens
    text = text.replace("-", " ")

    # Handle enumerations
    text = re.sub(r'\(i\)|\(ii\)|\(iii\)|\(iv\)|\(v\)|\(vi\)|\(vii\)|\(viii\)|\(ix\)|\(x\)', '', text)

    # Convert numbers to text
    text = ' '.join([p.number_to_words(word) if bool(re.search(r'\d', word)) else word for word in text.split()])

    # Remove punctuations
    text = re.sub(r'[^\w\s]', '', text)

    # Tokenize and remove stopwords
    tokens = word_tokenize(text)
    tokens = [word for word in tokens if word.lower() not in stopwords.words('english')]

    return ' '.join(tokens)

# Tokenize the text in df_train into sentences
sentences_train = [sent_tokenize(text) for text in df_train_n['text']]
sentences_train = [sentence for sublist in sentences_train for sentence in sublist]  # Flatten the list

# Apply the pre-processing function to each sentence
processed_sentences = [preprocess_text(sentence) for sentence in sentences_train]

processed_sentences[:5]  # Display first 5 processed sentences


['automatic question generation five one open domain indonesian questions using syntactical template based features academic textbooks measuring education quality school conducted delivering examination students',
 'composing questions examination process measure students achievement school teaching learning process difficult time consuming',
 'solve problem research proposes automatic question generation aqg method generate open domain indonesian question using syntactical approach',
 'open domain questions questions covering many domains knowledge',
 'challenge generating questions identify types declarative sentences potential transformed questions develop method generating question automatically']

In [ ]:
def check_for_numbers(processed_sentences):
    sentences_with_numbers = [sentence for sentence in processed_sentences if bool(re.search(r'\d', sentence))]
    return sentences_with_numbers

# Using the function
sentences_with_numbers = check_for_numbers(processed_sentences)

if sentences_with_numbers:
    print(f"Found {len(sentences_with_numbers)} sentences with numbers:")
    for sent in sentences_with_numbers:
        print(sent)
else:
    print("No numbers found in the processed sentences!")


No numbers found in the processed sentences!


In [ ]:
len(processed_sentences)

26755

In [ ]:
# # Function to apply all preprocessing steps
# def preprocess_text(text):
#     # Convert text to lowercase
#     text = text.lower()

#     # Remove URLs
#     text = re.sub(r'http\S+', '', text)

#     # Remove bullets and enumerations
#     text = re.sub(r'•|\d+\.', '', text)

#     # Remove apostrophes
#     text = re.sub(r"([’'‘’])", '', text)

#     # Remove hyphens
#     text = re.sub(r"-", ' ', text)

#     # Convert numerical digits to text
#     text = ' '.join([num2words(word) if word.isdigit() else word for word in text.split()])

#     # Tokenize the text
#     tokens = word_tokenize(text)

#     # Remove stopwords
#     stop_words = set(stopwords.words('english'))
#     tokens = [word for word in tokens if word.lower() not in stop_words]

#     # Remove punctuation
#     tokens = [word for word in tokens if word.isalnum()]

#     # Join tokens back into a single string
#     text = ' '.join(tokens)

#     return text

# # Apply the preprocessing function to the "text" column
# combined_df['text'] = combined_df['text'].apply(preprocess_text)

# # Print the preprocessed DataFrame
# print(combined_df)

In [ ]:
# Initialize the TF-IDF vectorizer
tfidf_vectorizer = TfidfVectorizer()

# Fit the vectorizer to the training data and transform it to create the context matrix
context_matrix_train = tfidf_vectorizer.fit_transform(processed_sentences)


In [ ]:
tfidf_feature_names = tfidf_vectorizer.get_feature_names_out()

In [ ]:
# Convert the TF-IDF context matrix to a DataFrame for easier viewing
context_matrix_train_df = pd.DataFrame(context_matrix_train.toarray(), columns=tfidf_vectorizer.get_feature_names_out())

# Display the DataFrame
print(context_matrix_train_df)

       ____________________________________________   aa  aaa  aaai  aaate  \
0                                               0.0  0.0  0.0   0.0    0.0   
1                                               0.0  0.0  0.0   0.0    0.0   
2                                               0.0  0.0  0.0   0.0    0.0   
3                                               0.0  0.0  0.0   0.0    0.0   
4                                               0.0  0.0  0.0   0.0    0.0   
...                                             ...  ...  ...   ...    ...   
26750                                           0.0  0.0  0.0   0.0    0.0   
26751                                           0.0  0.0  0.0   0.0    0.0   
26752                                           0.0  0.0  0.0   0.0    0.0   
26753                                           0.0  0.0  0.0   0.0    0.0   
26754                                           0.0  0.0  0.0   0.0    0.0   

       aab  aachen  aacr  aals  aalto  ...  ﬁnally  ﬁnd  ﬁnding

multiply matrix with hot vector

In [ ]:
from gensim.models import Word2Vec

# # Tokenize the sentences in your preprocessed training data
tokenized_sentences = [sentence.split() for sentence in processed_sentences]

# # Set Word2Vec hyperparameters
# vector_size = 100  # Adjust the vector size as needed
# window = 5  # Adjust the context window size as needed
# min_count = 1  # Minimum word count, adjust as needed
# sg = 0  # Use CBOW (Continuous Bag of Words) model, set to 1 for Skip-gram

# # Initialize and train the Word2Vec model
# word2vec_model = Word2Vec(tokenized_sentences, vector_size=vector_size, window=window, min_count=min_count, sg=sg)


In [ ]:
model = Word2Vec(sentences=tokenized_sentences, vector_size=100, window=5, min_count=1, workers=4)
model.train(tokenized_sentences, total_examples=len(tokenized_sentences), epochs=100)

(42525574, 43777600)

In [ ]:
def tfidf_weighted_word2vec_optimized(tfidf_matrix, tfidf_feature_names, w2v_model):
    # Precompute word embeddings for all the vocabulary words
    word_embeddings = {word: w2v_model.wv[word] for word in tfidf_feature_names if word in w2v_model.wv}

    # Convert word embeddings to a matrix form
    embedding_matrix = np.zeros((len(tfidf_feature_names), w2v_model.vector_size))
    for i, word in enumerate(tfidf_feature_names):
        if word in word_embeddings:
            embedding_matrix[i] = word_embeddings[word]

    # Use matrix multiplication to get the document embeddings
    doc_embeddings = tfidf_matrix.dot(embedding_matrix)

    return doc_embeddings

In [ ]:
doc_embeddings = tfidf_weighted_word2vec_optimized(context_matrix_train, tfidf_feature_names, model)
print(doc_embeddings)

[[ 2.94066365 -2.75991958  0.95635568 ...  2.65744452 -0.6219132
   0.9912207 ]
 [ 2.73634839 -1.58208441  2.62256372 ...  2.13562283 -3.08488217
   2.17031348]
 [ 0.54672254  1.95034883  1.82091232 ...  2.92333608 -3.38736684
   2.28553392]
 ...
 [ 1.09904305 -1.94454739 -0.98970658 ...  1.34743741 -0.63324706
   3.79796201]
 [ 0.28434048 -1.92747464 -0.99017892 ...  2.41962134  0.06745637
   2.52433333]
 [-0.93884438  1.52266418 -0.58592448 ... -0.51001998 -0.82599182
   4.80410532]]


In [ ]:
# Step 3: Create TF-IDF-weighted Word2Vec representations
# def tfidf_weighted_word2vec(tfidf_matrix, tfidf_feature_names, w2v_model):
#     doc_embeddings = []

#     for doc_tfidf in tfidf_matrix:
#         doc_embedding = np.zeros(w2v_model.vector_size)
#         for word, weight in zip(tfidf_feature_names, doc_tfidf.toarray()[0]):
#             if word in w2v_model.wv:
#                 doc_embedding += weight * w2v_model.wv[word]
#         doc_embeddings.append(doc_embedding)

#     return np.array(doc_embeddings)

# doc_embeddings = tfidf_weighted_word2vec(context_matrix_train, tfidf_feature_names, model)
# print(doc_embeddings)

next time to testing